# 寻找中位数
## 描述
两个有序数组，确定它们整体的中位数
## 解答
我们有两个递增数列$\{a_i\}(0\leq i<m)$和$\{b_i\}(0\leq i<n)$，他们合在一起排序后的数列为$\{c_i\}(0\leq i<t)$，其中$t=m+n$，显然最终的中位数可以表示为
$$
c=\frac{c_{\lfloor\frac{t-1}{2}\rfloor}+c_{\lceil\frac{t-1}{2}\rceil}}{2}.
$$
因此我们只需确定$\{c_i\}$中前$\lceil\frac{t-1}{2}\rceil$个元素便能确定$c$。我们需确定确定$\{c_i\}$中前$r$个元素。

### 思路1
$r$从0开始，逐一确定，此时$s,~k$分别表示这$r$个元素来自数列$\{a_i\}$和$\{b_i\}$的数量，显然时间复杂度为$O(m+n)$。

In [16]:
def findMedianSortedArrays(nums1, nums2):
    m, n = len(nums1), len(nums2)
    s, k = 0, 0
    totalSort = []
    m1 = ((m+n-1)-(m+n+1)%2) // 2
    m2 = ((m+n-1)+(m+n+1)%2) // 2
    while(s+k < m2+1):
        if s == m:
            totalSort.append(nums2[k])
            k += 1
            continue
        if k == n:
            totalSort.append(nums1[s])
            s += 1
            continue
        if nums1[s] < nums2[k]:
            totalSort.append(nums1[s])
            s += 1
        else:
            totalSort.append(nums2[k])
            k += 1
    return (totalSort[m1]+totalSort[m2]) / 2.0

### 思路2
我们先确定$\{c_i\}$中前$r=\lfloor\frac{t-1}{2}\rfloor$个元素，然后再确定$c_{\lfloor\frac{t-1}{2}\rfloor}$和$c_{\lceil\frac{t-1}{2}\rceil}$。比较$a_s$和$b_k$，其中$s+k=r-1$，若$a_s<b_k$，则$\{a_i\}(i\leq s)$必定在$\{c_i\}$前$r$个位置，因为$s+1+k=r$。时间复杂度为$O(\log(m+n)$。

In [12]:
def findMedianSortedArrays(nums1, nums2):
    t, cr = len(nums1) + len(nums2), 0
    r = ((t-1)-(t+1)%2) // 2
    while cr < r:
        rr = r - cr
        if len(nums1) < len(nums2):
            nums1, nums2 = nums2, nums1
        if len(nums2) == 0:
            nums1 = nums1[rr:]
            break
        k = min(len(nums2)-1, rr//2)
        s = rr - 1 - k
        if nums1[s] < nums2[k]:
            cr += (s+1)
            nums1 = nums1[s+1:]
        else:
            cr += (k+1)
            nums2 = nums2[k+1:]

    nums1 += [float('inf'),float('inf')]
    nums2 += [float('inf'),float('inf')]
    median = sorted([nums1[0],nums1[1],nums2[0],nums2[1]])[:(t+1)%2+1]
    return (median[0]+median[(t+1)%2]) / 2
    

### 另一种更简便的解法
当$r=\lceil\frac{t-1}{2}\rceil=\lfloor\frac{t}{2}\rfloor$时，$s,~k$分别表示这$r$个元素来自数列$\{a_i\}$和$\{b_i\}$的数量，此时数列$\{c_i\}$可以写为$\{a_0,\cdots,a_{s-1},b_0,\cdots,b_{k-1}~|~a_s,\cdots,a_{m-1},b_k,\cdots,b_{n-1}\}$，`|`右侧元素大于或等于左侧的元素，即$b_{k-1}\leq a_s$且$a_{s-1}\leq b_k$。采用二分搜索法，搜索步长逐次减半，若$a_{s-1}>b_k$，向左搜索$s$，若$b_{k-1}>a_s$时，向右搜索$s$。

最终，当$t$为偶数时，$c=\frac{1}{2}[\max(a_{s-1},b_{k-1})+\min(a_s,b_k)]$，当$t$为奇数时，$c=\min(a_s,b_k)$。

该方法时间复杂度为$O(\log_2(\min(m,n)))$

In [19]:
def findMedianSortedArrays(nums1, nums2):
    if (len(nums1)>len(nums2)):      #确保nums1的长度不大于nums2
        nums1, nums2 = nums2, nums1
    I = float('inf')
    nums1, nums2 = [-I] + nums1 + [I], [-I] + nums2 + [I] # 确保两个数组都不为空
    r = (len(nums1) + len(nums2)) // 2
    s, k, step = 1, r-1, (len(nums1) + 1) // 2
    while not (nums2[k-1] <= nums1[s] and nums1[s-1] <= nums2[k]): # 二分搜索
        step = (step + 1) // 2  #向上取整
        if nums2[k-1] > nums1[s]:
            s +=  step
        else:
            s -=  step
        k = r - s
    
    if (len(nums1) + len(nums2)) % 2:
        return min(nums1[s], nums2[k])
    else:
        return (max(nums1[s-1],nums2[k-1]) + min(nums1[s], nums2[k])) * 0.5